In [17]:
import keras
from keras.applications import ResNet50
from keras.models import Sequential, load_model, Model
from keras import models, optimizers
from keras.optimizers import SGD
from keras.preprocessing.image import load_img, img_to_array
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from IPython.display import SVG
import pydot
import os
import glob
import numpy as np
import pandas as pds
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.normalization import BatchNormalization as BN
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_diabetes
import re

In [68]:
# 正規化 numpyの配列にして平均0分散1で返す {画像パス : 数値}の配列を引数にとる
def normalization(dict):
    # 分散・平均をとる
    values = [i for i in dict.values()]
    values = np.array(values)
    value_mean = values.mean()
    value_std  = np.std(values)
    print("mean = {}, std= {}".format(value_mean, value_std))
    # 要素を正規化
    for k, v in dict.items():
        dict[k] = ((v - value_mean)/value_std).astype(np.float32)
    print(sum(dict.values()))

In [71]:
# 全体の5%トリムを行うセル

# ディクショナリの最大もしくは最小の要素を削除
def del_dict_max_or_min(dict, max_or_min):
    if max_or_min is "max":
        key = max(dict, key=dict.get)
    else:
        key = min(dict, key=dict.get)
    del dict[key]
    
# 5%トリム {画像パス : 数値}の配列を引数にとる
def trim5(dict):
    dict_size = len(dict)
    half_trim_size = int(dict_size*25/1000)
    [del_dict_max_or_min(dict, "max") for _ in range(half_trim_size)]
    [del_dict_max_or_min(dict, "min") for _ in range(half_trim_size)]

In [89]:
img_path_list = glob.glob("./image/valu_cutout_face_over64/*.jpg")
dataset = {}
for img_path in img_path_list:
    # 画像のパス
    img = img_path
    # ファイル名から時価総額をfloat取得
    file_name = img_path.replace("./image/valu_cutout_face_over64/", "")
    file_name = re.sub("__.+__", "", file_name)
    file_name = re.sub(".jpg", "", file_name)
    file_name = file_name.replace(",", "")
    file_name = float(file_name)
    dataset[img] = file_name

trim5(dataset)
#normalization(dataset)
print(len(dataset))

5548


In [90]:
# 548をテストデータに5000を学習データにする
test_data = {}
for i in range(548):
    k, v = dataset.popitem()
    test_data[k] = v
train_data = dataset

548
5000
{'./image/valu_cutout_face_over64/9.990000__0.11209319348130398__.jpg': 9.99, './image/valu_cutout_face_over64/0.150000__0.021879622110240082__.jpg': 0.15, './image/valu_cutout_face_over64/0.444000__0.5689738414314879__.jpg': 0.444, './image/valu_cutout_face_over64/1.586000.jpg': 1.586, './image/valu_cutout_face_over64/0.434000__0.19019612531550079__.jpg': 0.434, './image/valu_cutout_face_over64/0.131000__0.5882349139677454__.jpg': 0.131, './image/valu_cutout_face_over64/1.631000.jpg': 1.631, './image/valu_cutout_face_over64/0.590000__0.9637681754499915__.jpg': 0.59, './image/valu_cutout_face_over64/2.261000__0.24745279570530743__.jpg': 2.261, './image/valu_cutout_face_over64/6.500000.jpg': 6.5, './image/valu_cutout_face_over64/0.430000__0.024110029840059832__.jpg': 0.43, './image/valu_cutout_face_over64/5.000000__0.2646097049904482__.jpg': 5.0, './image/valu_cutout_face_over64/1.486000.jpg': 1.486, './image/valu_cutout_face_over64/0.182000__0.400587422457947__.jpg': 0.182, '.

In [127]:
# 学習用のデータセットをリストに分解してndarray型に変更
def preprocessing_img224(img_path_list):
    img_list = []
    for img_path in img_path_list:
        row_img = load_img(img_path, target_size=(224,224))
        img_array = img_to_array(row_img)
        img_array /= 255.0
        img_list.append(img_array)
    return img_list

In [177]:
def two_dimencional_array_label(dict_datas):
    labels = []
    for i in dict_datas.values():
        a = [i]
        a = np.array(i, dtype="float32")
        labels.append(a)
    return labels

In [178]:
# 学習用のデータセットをディクショナリから分離して、画像と値をそれぞれをリストにして前処理する
train_img_path = [i for i in train_data.keys()]
train_img = preprocessing_img224(train_img_path)

train_label = two_dimencional_array_label(train_data)

In [134]:
# テスト用のデータセットをディクショナリから分離して、画像と値をそれぞれをリストにして前処理する
test_img_path = [i for i in test_data.keys()]
test_img = preprocessing_img224(test_img_path)

test_label = two_dimencional_array_label(test_data)

In [ ]:
# list2ndarray
train_img = np.array(train_img)
train_label = np.array(train_label)
test_img = np.array(test_img)
test_label = np.array(test_label)

In [136]:
print("{}, {}".format(train_img[0].shape, train_label[0]))
print(len(test_img))
print(len(test_label))

(224, 224, 3), 9.989999771118164
548
548


In [2]:
# ResNet50のインポート ファインチューニング
def create_model():
    resnet_model = ResNet50(include_top=False, weights="imagenet", input_tensor=Input(shape=(224, 224, 3)))

    dense  = Flatten()(Dense(2048, activation='relu')(BN()(resnet_model.layers[-1].output)))
    result = Dense(1)(dense)
  
    model = Model(inputs=resnet_model.input, outputs=result)
    
    for layer in model.layers[:80]: # default 179
        if 'BatchNormalization' in str(layer):
          pass
        else:
          layer.trainable = False
    model.compile(loss='mse', optimizer='adam')
    return model

In [3]:
model = create_model()

/usr/local/var/pyenv/versions/3.6.0/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [18]:
plot_model(model, to_file="my_model.png", show_shapes=True)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [7]:
estimator = KerasRegressor(build_fn=create_model, epochs=100, batch_size=256, verbose=1)

In [186]:
estimator.fit(x=train_img, y=train_label)
#img_pred = estimator.predict(test_img)
#mse = mean_squared_error(test_label, img_pred)
#print("KERAS REG RMSE : %.2f" % (mse ** 0.5))

/usr/local/var/pyenv/versions/3.6.0/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/100


KeyboardInterrupt: 